In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

#载入数据集,从网络中获取，使用one_hot来将标签变为0/1格式
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)


#每个批次大小
batch_size = 50
#计算一共有多少批次  //为整除
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_sumaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var) 
        tf.summary.scalar("mean",mean) #平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev",stddev) #标准差
        tf.summary.scalar('max',tf.reduce_max(var)) #最大值
        tf.summary.scalar("min",tf.reduce_min(var)) #最小值
        tf.summary.histogram("histogram",var) #直方图

#命名空间
with tf.name_scope("input"):
    #定义两个占位符
    x = tf.placeholder(tf.float32,[None,784],name="x-input")
    y = tf.placeholder(tf.float32,[None,10],name="y-input")

with tf.name_scope("layer"):
    with tf.name_scope("weight"):
        #创建一个简单的神经网络，只包含输入层和输出层
        W = tf.Variable(tf.zeros([784,10]),name='W')  #权值
        variable_sumaries(W)
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]),name='b')      #偏置
        variable_sumaries(b)
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)  #将信号总和转换为概率值

with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar("loss",loss)
with tf.name_scope("train"):
    #使用梯度下降法训练
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量                
init = tf.global_variables_initializer()

with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        #比较两个参数是否相同，返回布尔型列表，用于检测训练效果,argmax函数用于求最大概率的位置（标签），返回以为张量中最大值所再位置
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  
    with tf.name_scope("accuracy"):
        #求准确率,cast将对比的布尔型转换为浮点型，然后求平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar("accuracy",accuracy)
        
#合并所有的summary
merged = tf.summary.merge_all()
                
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("E:/Tensorflow/logs",sess.graph)
    for epoch in range(21):  #把所有的训练21次
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)   #把训练集中的图片循环一次
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
        writer.add_summary(summary,epoch) #将summary运行结果写入文件       
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch)+ ",Test Accuracy" + str(acc))       

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter0,Test Accuracy0.8939
Iter1,Test A